# Simulateur de drone


Un drone quadriotor ou quadricoptère peut effectuer quatre mouvements:
<ul>
    <li> Les gaz : permettent d'effectuer une montée ou une descente en altitude </li>
    <li> Le lacet: permet au drone d'effectuer une rotation sur lui même dans le sens des aiguilles d'une montre ou dans le sens inverse </li>
    <li>  le rouli: permet au drone de se pencher sur un de ses axes pour effectuer une translation à gauche ou à droite  </li>
    <li>  le tangage : permet au drone de se pencher sur un de ses axes pour avancer ou reculer</li>
</ul>
    

<a><img src="https://preview.ibb.co/gsEZck/Capture_d_cran_2017_06_23_04_12_11.png" width=100% alt="Capture_d_cran_2017_06_23_04_12_11" border="0"></a>

Il y a aussi biensur une commande décoller et une commande attérir. Lorsqu'on utilise un programme pour piloter le drone, il faudra à chaque fois préciser la vitesse et la durée lors de l'envoi des commandes.

## Paramétrage


soit OXYZ le repère orthonormé fixe lié au sol et AX'Y'Z' le repère orthonormé mobile lié au drone où A est le centre de gravité du drone.
Nous travaillons dans un répère cylindrique. Ainsi les directions OZ et OZ' coincident à tout moment.
Pour comprendre le paramètrage en terme d'angle, nous allons considérer un point B. Le drone (en A) est censé aller en B.
<img src="https://preview.ibb.co/kDyXV5/Capture_d_cran_2017_06_23_04_29_21.png" alt="Capture_d_cran_2017_06_23_04_29_21" width=100% border="0">
<img src="https://preview.ibb.co/kfoGq5/Capture_d_cran_2017_06_23_04_31_20.png" alt="Capture_d_cran_2017_06_23_04_31_20" width=100% border="0">

## Le modèle

Dans la suite, on va modéliser le drone par une classe objet avec des attributs de position et de vitesse maximale. Les commandes de base seront implémentées tout simplement par des méthodes qui mettront à jour la position du drone.

### Test du planificateur de vol en mode "translations only"

In [ ]:
import random


X1=[100,200,250]
Y1=[100,200,300]
Z1=[350,400,280]


for i in range(15):
    X1 += [X1[-1] + int(random.uniform(0,200))]
    Y1 += [Y1[-1] + int(random.uniform(0,200))]
    Z1 += [Z1[-1] + int(random.uniform(0,200))]
for i in range(10):
    X1 += [X1[-1] + int(random.uniform(0,200))]
    Y1 += [Y1[-1] + int(random.uniform(0,200))]
    Z1 += [Z1[-1] - int(random.uniform(0,200))]




monDrone = Drone()
monDrone.takeoff()
pas=50

for i in range(len(X1)):
    monDrone.goToPoint1([X1[i], Y1[i], Z1[i]], pas)

X = monDrone.goneToX
Y = monDrone.goneToY
Z = monDrone.goneToZ

X2 = [ monDrone.positionX[i] for i in range (1, len(monDrone.positionX), 1000)]
Y2 = [ monDrone.positionY[i] for i in range (1, len(monDrone.positionY), 1000)]
Z2 = [ monDrone.positionZ[i] for i in range (1, len(monDrone.positionZ), 1000)]
del monDrone

print(X1)
print(Y1)
print(Z1)
print(X)
print(Y)
print(Z)
print(Y2)


tracerTrajectoire(X,Y, Z, X1, Y1, Z1)
animerPoints(X2,Y2,Z2)



<img src="https://image.ibb.co/dBXRq5/Capture_d_cran_2017_06_23_04_32_50.png" alt="Capture_d_cran_2017_06_23_04_32_50" width=100% border="0">

## Estimation de l'erreur

On peut estimer en calculant la distance entre les points de la trajectoire théorique et les points de la trajectoire réelle du drone. Même dans l’hypothèse où le drone parcourait tous les points de la trajectoire théorique imposées, la trajectoire du drone enregistré avec la mocap paraitra continu (densité de points élevée). Il faudra alors, re-échantillonner la trajectoire réelle du drone de façon à obtenir un même nombre de points que celui de la trajectoire théorique.

L'erreur reste quasiment nulle si la trajectoire théorique est un polygone dont les sommets sont sufisaments espacés.

#### Re-échantillonage des trajecoires pour avoir un meme nombre de points

In [17]:
import math

def distance(point1, point2):
    return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2 +  (point1[2]-point2[2])**2)


def alignerEchantillon(pointsTheoriques, pointsMesures) :
    nouvelleSuite=[[],[],[]]
    erreurs = []
    d=0
    s = distance([pointsMesures[0][0], pointsMesures[1][0], pointsMesures[2][0]],
                 [pointsTheoriques[0][d], pointsTheoriques[1][d], pointsTheoriques[2][d]])
    i=1
    while i < len(pointsMesures[0]) and d < len(pointsTheoriques[0]):

        m=distance([pointsMesures[0][i],pointsMesures[1][i], pointsMesures[2][i]], [pointsTheoriques[0][d], pointsTheoriques[1][d], pointsTheoriques[2][d]])
        if m>=s :
            nouvelleSuite[0]+= [pointsMesures[0][i]]
            nouvelleSuite[1]+= [pointsMesures[1][i]]
            nouvelleSuite[2]+= [pointsMesures[2][i]]
            erreurs += [s]
            d+=1
            if d==len(pointsTheoriques[0]): break
            s = distance([pointsMesures[0][i], pointsMesures[1][i], pointsMesures[2][i]],
                         [pointsTheoriques[0][d], pointsTheoriques[1][d], pointsTheoriques[2][d]])
        else:
            s=m
        i+=1
    return nouvelleSuite, erreurs


Dans l’exemple de la figure ci-dessous, on ne retiendra que les points en vert de la trajectoire rouge (plus dense en point que la trajectoire bleue). Il s’agit en fait des projections (approximatives) orthogonales des points bleus sur la trajectoire rouge.


<img src="https://preview.ibb.co/hv96q5/Capture_d_cran_2017_06_23_04_34_16.png" alt="Capture_d_cran_2017_06_23_04_34_16" border="0">

#### Test 1

In [ ]:
X1=[100,200,250]
Y1=[100,200,300]
Z1=[350,400,280]


for i in range(15):
    X1 += [X1[-1] + int(random.uniform(0,200))]
    Y1 += [Y1[-1] + int(random.uniform(0,200))]
    Z1 += [Z1[-1] + int(random.uniform(0,200))]
for i in range(10):
    X1 += [X1[-1] + int(random.uniform(0,200))]
    Y1 += [Y1[-1] + int(random.uniform(0,200))]
    Z1 += [Z1[-1] - int(random.uniform(0,200))]




monDrone = Drone()
monDrone.takeoff()
pas=200

for i in range(len(X1)):
    monDrone.goToPoint1([X1[i], Y1[i], Z1[i]], pas)
    
X = monDrone.positionX
Y = monDrone.positionY
Z = monDrone.positionZ

X, erreurs = alignerEchantillon([X1, Y1, Z1], [X, Y, Z])
print(len(X[0])-len(X1))
tracerTrajectoire(X[0], X[1],X[2], X1, Y1, Z1)
plt.plot(erreurs)
plt.show()

print("erreur moyen", sum(erreurs)/len(erreurs))
print("erreur max", max(erreurs))




<img src="https://image.ibb.co/eK4pA5/Capture_d_cran_2017_06_23_04_00_49.png" alt="Capture_d_cran_2017_06_23_04_00_49" border="0">

#### Test 2

In [ ]:
from numpy import *

theta=linspace(-16*pi,16*pi,800) 
z=linspace(-2,2,800)
r=z*z+1
x=r*sin(theta)
y=r*cos(theta)

z=z*1000

X1=x.tolist()
Y1=y.tolist()
Z1=z.tolist()


monDrone = Drone()
monDrone.takeoff()
pas=30

for i in range(len(X1)):
    monDrone.goToPoint1([X1[i], Y1[i], Z1[i]], pas)
monDrone.land()

X = monDrone.goneToX
Y = monDrone.goneToY
Z = monDrone.goneToZ


X, erreurs = alignerEchantillon([X1, Y1, Z1], [X, Y, Z])
print(len(X[0])-len(X1))
tracerTrajectoire(X[0], X[1],X[2], X1, Y1, Z1)
plt.plot(erreurs)
plt.show()

print("erreur moyenne", sum(erreurs)/len(erreurs))
print("erreur max", max(erreurs))



<img src="https://image.ibb.co/cYuxV5/Capture_d_cran_2017_06_23_04_01_23.png" alt="Capture_d_cran_2017_06_23_04_01_23" border="0">

#### Fichier executable pour le drone

In [ ]:
#generation du fichier nodeJs pour le drone

suitecommandes="var arDrone = require('ar-drone');\nvar client  = arDrone.createClient();\n \n client.takeoff(); \n "
suitecommandes+=planning

fichierVol = open("fichierVol.js", "w")
fichierVol.write(suitecommandes)
fichierVol.close()

On obtient un fichier de cette forme

<img src="https://image.ibb.co/fveaHk/Capture_d_cran_2017_06_23_02_42_29.png" alt="Capture_d_cran_2017_06_23_02_42_29" border="0">